In [1]:
import numpy as np # used for arrays
import gym # pull the environment
import time # to get the time
import math # needed for calculations
import gym
from gym import logger as gymlogger
from gym.wrappers import RecordVideo
gymlogger.set_level(40) #error only
# import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
# %matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
import pickle


In [2]:
class QLearning:
    def __init__(self):
        self.env = gym.make('CartPole-v1')
        # q_table = None
        # self.create_q_table()

        self.p_space = np.linspace(-2.4, 2.4, 10)
        self.v_space = np.linspace(-4, 4, 10)
        self.a_space = np.linspace(-.2095, .2095, 10)
        self.av_space = np.linspace(-4, 4, 10)

        self.alpha = 0.1
        self.gamma = 0.99
        self.epsilon = 1
        self.epsilon_decay_rate = 0.00001
        
        # training variables
        # self.no_of_Episodes = 100
        self.is_training = True
        self.end = False
        self.train_episode_count = 0
        self.reward_training_episodes = []

    #train agent and update q values
    def trainingEpisodes(self):
       
        q_table = np.zeros((len(self.p_space)+1, 
                      len(self.v_space)+1, 
                      len(self.a_space)+1, 
                      len(self.av_space)+1, 
                      self.env.action_space.n)) # init a 11x11x11x11x2 q table array with values set to 0

        while(True):
            # initialise everything
            # print(" =========== training round "+str(self.train_episode_count) + " ===========")
            
            #initialises the environment
            current_state = self.env.reset()[0]       
            self.end = False
            rewards = 0
            
            #convert the contininous state into discrete bin values for q table to compute 
            state_p = np.digitize(current_state[0],self.p_space)
            state_v= np.digitize(current_state[1],self.v_space)
            state_a=np.digitize(current_state[2],self.a_space)
            state_av=np.digitize(current_state[3],self.av_space)

            # print(" == starting state: ==")
            # print(current_state)
            # print(" == starting index: ==")
            # print(state_index)
            
            #close loop when either episode ends or rewards exceeds desired threshold
            while(not self.end and rewards<1000):
                
                #generates random number to determine whether to explore or exploit
                randomNumber=np.random.random()
                if randomNumber < self.epsilon:
                    #selecting a random action
                    action = self.env.action_space.sample()
                else:
                    #selecting action with the largest q value
                    action = np.argmax(q_table[state_p, state_v, state_a, state_av, :])

                #executes the action and determines the new state,rewards obtained and whether episode would end
                new_state, reward, self.end, _ ,_ = self.env.step(action)
                
                #convert the next contininous state into discrete bin values for q table to compute 
                new_state_p = np.digitize(new_state[0],self.p_space)
                new_state_v= np.digitize(new_state[1],self.v_space)
                new_state_a=np.digitize(new_state[2],self.a_space)
                new_state_av=np.digitize(new_state[3],self.av_space)
                                
                #updates q table values                
                q_table[state_p, state_v, state_a, state_av, action] = q_table[state_p, state_v, state_a, state_av, action] + self.alpha * (reward + self.gamma * np.max(q_table[new_state_p, new_state_v, new_state_a, new_state_av,:]) - q_table[state_p, state_v, state_a, state_av, action])
                current_state = new_state
                state_p = new_state_p
                state_v = new_state_v
                state_a = new_state_a
                state_av= new_state_av
                rewards += reward

            #adds the reward obtained in the current episode to list containing rewards obtained in each episode
            self.reward_training_episodes.append(rewards)
            
            #calcalates mean rewards obtained over the past 100 epsiodes
            mean_rewards = np.mean(self.reward_training_episodes[len(self.reward_training_episodes)-100:])
            
            
            #prints out training updates every 100 episodes
            if self.train_episode_count%100==0:
                print(f'Episode: {self.train_episode_count}: {rewards}  Epsilon: {self.epsilon:0.2f}  Mean_Rewards {mean_rewards:0.1f}')
            
            #end training if satisfactory mean rewards values above 500 obtained
            if mean_rewards>500:
                break

            #decreases exploration rate so agent will lean towards exploitation over time    
            self.epsilon = max(self.epsilon - self.epsilon_decay_rate, 0)
            
            #increment training episode
            self.train_episode_count+=1

        #close environment
        self.env.close()
        
        #saves q table
        if self.is_training:
            f = open('cartpole.pkl','wb')
            pickle.dump(q_table, f)
            f.close()
    
    #simulate episodes without updating q values
    def simulateEpisodes(self):
        
        #create environment
        self.env = gym.make('CartPole-v1', render_mode='human')
        
        #loads q table
        f = open('cartpole.pkl', 'rb')
        q_table = pickle.load(f)
        f.close()
        
 

        while(True):
            # initialise everything
            # print(" =========== training round "+str(self.train_episode_count) + " ===========")
            
            #initalises the environment
            self.train_episode_count = 0
            self.reward_training_episodes = []
            current_state = self.env.reset()[0]       
            self.end = False
            rewards = 0
            
            
            #convert the state into discrete bin values
            state_p = np.digitize(current_state[0],self.p_space)
            state_v= np.digitize(current_state[1],self.v_space)
            state_a=np.digitize(current_state[2],self.a_space)
            state_av=np.digitize(current_state[3],self.av_space)

            #close loop when either episode ends or rewards exceeds desired threshold
            while(not self.end and rewards<1000):
                
                #selecting action with the largest q value
                action = np.argmax(q_table[state_p, state_v, state_a, state_av, :])
                
                #convert the next contininous state into discrete bin values for q table to compute 
                new_state, reward, self.end, _ ,_ = self.env.step(action)
                new_state_p = np.digitize(new_state[0],self.p_space)
                new_state_v= np.digitize(new_state[1],self.v_space)
                new_state_a=np.digitize(new_state[2],self.a_space)
                new_state_av=np.digitize(new_state[3],self.av_space)
                                
                #transitions agent to the next state
                current_state = new_state
                state_p = new_state_p
                state_v = new_state_v
                state_a = new_state_a
                state_av= new_state_av
                rewards += reward

            #adds the reward obtained in the current episode to list containing rewards obtained in each episode
            self.reward_training_episodes.append(rewards)
            
            #calcalates mean rewards obtained over the past 100 epsiodes
            mean_rewards = np.mean(self.reward_training_episodes[len(self.reward_training_episodes)-100:])
            
            #prints out updates every 100 episodes
            if self.train_episode_count%100==0:
                print(f'Episode: {self.train_episode_count}: {rewards}  Epsilon: {self.epsilon:0.2f}  Mean_Rewards {mean_rewards:0.1f}')
            
            #end episode if satisfactory mean rewards values above 200 obtained
            if mean_rewards>200:
                break
            
             #increment training episode
            self.train_episode_count+=1

        #close the environment
        self.env.close()
        


        

In [3]:
a = QLearning()

a.simulateEpisodes()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode: 0: 601.0  Epsilon: 1.00  Mean_Rewards 601.0
